In [1]:
from philaudit.extractor import Extractor
import pandas as pd
import os

In [2]:
metadata = pd.read_csv(
    "/Users/jackvaughan/Downloads/PhilAuditStorage/Metadata/2013_metadata.csv"
)

In [3]:
metadata

,document,year,city_or_municipality,region,barangay,path,identifier,pages,pg_count,pageless
0,Tantangan2013_Audit_Report.pdf,2013,Municipalities,Region_XII,South_Cotabato,/Users/jackvaughan/Downloads/PhilAuditStorage/...,Region_XII_South_Cotabato_Tantangan2013_Audit_...,"34, 36",2,False
1,11-LakeSebu2013_Status_of_PY's_Recommendations...,2013,Municipalities,Region_XII,South_Cotabato,/Users/jackvaughan/Downloads/PhilAuditStorage/...,Region_XII_South_Cotabato_11-LakeSebu2013_Stat...,1-3,3,False
2,01-Surallah2013_Audit_Report.pdf,2013,Municipalities,Region_XII,South_Cotabato,/Users/jackvaughan/Downloads/PhilAuditStorage/...,Region_XII_South_Cotabato_01-Surallah2013_Audi...,NaN,0,True
3,01-Sto.Niño2013_Audit_Report.pdf,2013,Municipalities,Region_XII,South_Cotabato,/Users/jackvaughan/Downloads/PhilAuditStorage/...,Region_XII_South_Cotabato_01-Sto.Niño2013_Audi...,23-25,3,False
4,11-Tboli2013_Status_of_PY's_Recommendations.pdf,2013,Municipalities,Region_XII,South_Cotabato,/Users/jackvaughan/Downloads/PhilAuditStorage/...,Region_XII_South_Cotabato_11-Tboli2013_Status_...,1-2,2,False
...,...,...,...,...,...,...,...,...,...,...
1604,01-SanCarlosCity2013_Audit_Report.pdf,2013,Cities,Region_VI,Negros_Occidental,/Users/jackvaughan/Downloads/PhilAuditStorage/...,Region_VI_Negros_Occidental_01-SanCarlosCity20...,44-47,4,False
1605,10-BagoCity2013_Part3-Status_of_PY's_Recommend...,2013,Cities,Region_VI,Negros_Occidental,/Users/jackvaughan/Downloads/PhilAuditStorage/...,Region_VI_Negros_Occidental_10-BagoCity2013_Pa...,1-3,3,False
1606,11-CadizCity2013_Part3-Status_of_PY's_Recommen...,2013,Cities,Region_VI,Negros_Occidental,/Users/jackvaughan/Downloads/PhilAuditStorage/...,Region_VI_Negros_Occidental_11-CadizCity2013_P...,1-14,14,False
1607,04-TalisayCity2013_Part3-Status_of_PY's_Recomm...,2013,Cities,Region_VI,Negros_Occidental,/Users/jackvaughan/Downloads/PhilAuditStorage/...,Region_VI_Negros_Occidental_04-TalisayCity2013...,1-7,7,False


In [4]:
pas = "/Users/jackvaughan/Downloads/PhilAuditStorage"
root = pas + "2013"
year = "2013"
metadata_path = pas + "Metadata"
df = metadata
out = os.path.join(pas, "Extracted", year)

In [5]:
import os
import shutil
import sys
from pathlib import Path

import pandas as pd
from tqdm import tqdm

from philaudit.extractor import Extractor


def handle_existing_file(md_acc, path):
    if os.path.exists(path):
        md_acc.append((False, None, path))
        return True
    return False


def handle_pageless(md_acc, row, pageless_dir):
    if row.pageless:
        pageless_path = os.path.join(pageless_dir, row.document)
        if handle_existing_file(md_acc, pageless_path):
            return True

        md_acc.append((True, None))
        shutil.copy(row.path, pageless_dir)
        return True
    return False


def handle_errors(md_acc, row, extractor, no_lattice_dir, image_pdfs_dir):
    if extractor.table_extractor.no_lattice:
        nl_path = os.path.join(no_lattice_dir, row.document)
        if handle_existing_file(md_acc, nl_path):
            return True
        md_acc.append((True, "Document has no lattice tables.", nl_path))
        shutil.copy(row.path, no_lattice_dir)
        return True

    if extractor.table_extractor.is_image_based:
        ipdf_path = os.path.join(image_pdfs_dir, row.document)
        if handle_existing_file(md_acc, ipdf_path):
            return True
        md_acc.append((True, "Document is image based. Missing some pages", ipdf_path))
        shutil.copy(row.path, image_pdfs_dir)
        return True

    return False


def extract_data(df, out):
    pageless_dir = os.path.join(out, "Errors", "Pageless")
    no_lattice_dir = os.path.join(out, "Errors", "No_lattice")
    image_pdfs_dir = os.path.join(out, "Errors", "Image_pdfs")
    complete_dir = os.path.join(out, "Complete")

    # initialize metadata accumulator to later append to df
    md_acc = []  # (error, error_msg, extracted_path)
    for _, row in tqdm(
        df.iterrows(),
        desc="Extracting data",
    ):
        complete_path = os.path.join(complete_dir, f"{row.identifier}.csv")
        if handle_existing_file(md_acc, complete_path):
            continue

        if handle_pageless(md_acc, row, pageless_dir):
            continue

        extractor = Extractor(row.path, row.pages)

        if handle_errors(md_acc, row, extractor, no_lattice_dir, image_pdfs_dir):
            continue

        try:
            extractor.doctable.to_csv(complete_path, index=False)
        except Exception as e:
            md_acc.append((True, e, None))
            print(f"Error with {row.document}, skipping. Lost {row.pg_count} pages.")
            continue

        md_acc.append((False, None, complete_path))

    df["error"] = [d[0] for d in md_acc]
    df["error_msg"] = [d[1] for d in md_acc]
    df["extracted_path"] = [d[2] for d in md_acc]
    return df

In [6]:
def go():
    md = extract_data(df, out)
    print("Done! Data extracted.")
    md.to_csv(os.path.join(metadata_path, f"{year}_metadata.csv"))
    print("Metadata has been updated with errors and extracted paths.")

In [7]:
go()

Extracting data: 410it [37:57,  5.22s/it]INFO:philaudit.table_extractor: /Users/jackvaughan/Downloads/PhilAuditStorage/2013/Municipalities/Region_III/Bulacan/11-Guiguinto2013_Part3-Status_of_PY's_Recommendations.pdf has no lattice tables. Flagging.
Extracting data: 421it [38:50,  4.83s/it]INFO:philaudit.table_extractor: /Users/jackvaughan/Downloads/PhilAuditStorage/2013/Municipalities/Region_III/Bulacan/13-Plaridel2013_Part3-Status_of_PY's_Recommendations.pdf has no lattice tables. Flagging.
Extracting data: 481it [1:29:00,  6.76s/it] INFO:philaudit.table_extractor: /Users/jackvaughan/Downloads/PhilAuditStorage/2013/Municipalities/Region_V/Camarines_Sur/10-Calabanga2013_Status_of_Implementations_of_ PY's_Recommendations.pdf has no lattice tables. Flagging.
Extracting data: 514it [1:32:06,  7.16s/it]INFO:philaudit.table_extractor: /Users/jackvaughan/Downloads/PhilAuditStorage/2013/Municipalities/Region_V/Albay/10-Bacacay2013_Status_of_PY's_Recommendations.pdf has no lattice tables. Flag

IndexError: tuple index out of range